In [71]:
import pandas as pd
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt
from src import support as sp
from sklearn.experimental import enable_iterative_imputer  
from sklearn.impute import IterativeImputer

#pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

df_flight = pd.read_csv('../Files/Customer Flight Activity.csv')
df_loyalty = pd.read_csv('../Files/Customer Loyalty History.csv')

In [72]:
# REMOVING DUPLICATES IF ANY

df_flight = sp.remove_duplicate_rows(df_flight)

print('--------')

df_loyalty = sp.remove_duplicate_rows(df_loyalty)

There were 405624 rows, and 1864 duplicated rows were deleted.
New row count: 403760 rows
--------
There were 16737 rows, and 0 duplicated rows were deleted.
New row count: 16737 rows


In [73]:
# FLOAT TO INTEGER LOYALTY

l_convert_to_int = ['Cancellation Year', 'Cancellation Month']


df_loyalty = sp.float_to_integer (df_loyalty, l_convert_to_int)
df_loyalty.dtypes # CONFIRMATION

Loyalty Number          int64
Country                object
Province               object
City                   object
Postal Code            object
Gender                 object
Education              object
Salary                float64
Marital Status         object
Loyalty Card           object
CLV                   float64
Enrollment Type        object
Enrollment Year         int64
Enrollment Month        int64
Cancellation Year       Int64
Cancellation Month      Int64
dtype: object

In [74]:
# REMOVING NEGATIVE VALUES FROM SALARY

df_loyalty['Salary'] = df_loyalty['Salary'].abs()

print('Negative values in Salary are now:' ,(df_loyalty['Salary']  < 0).sum())


Negative values in Salary are now: 0


In [75]:
# FILLING NULLS IN SALARY WITH ITERATIVE IMPUTER FOR MORE

imputer = IterativeImputer(random_state=42) #RANDON_STATE SO THE RANDON IMPUTER WONT GENERATE A DIFFERENT IMPUTER WHEN EXECUTED

df_loyalty['Salary'] = imputer.fit_transform(df_loyalty[['Salary']]) 

print('Nulls in Salary are now: ', df_loyalty['Salary'].isna().sum())


Nulls in Salary are now:  0


In [81]:
# REVIEWING DESCRIPTIVE
df_loyalty.describe().T

,count,mean,std,min,25%,50%,75%,max
Loyalty Number,16737.0,549735.880445,258912.132453,100018.0,326603.0,550434.0,772019.0,999986.0
Salary,16737.0,79359.340907,30029.311812,9081.0,63899.0,79359.340907,82940.0,407228.0
CLV,16737.0,7988.896536,6860.98228,1898.01,3980.84,5780.18,8940.58,83325.38
Enrollment Year,16737.0,2015.253211,1.979111,2012.0,2014.0,2015.0,2017.0,2018.0
Enrollment Month,16737.0,6.669116,3.398958,1.0,4.0,7.0,10.0,12.0
Cancellation Year,2067.0,2016.503145,1.380743,2013.0,2016.0,2017.0,2018.0,2018.0
Cancellation Month,2067.0,6.962748,3.455297,1.0,4.0,7.0,10.0,12.0


In [77]:
# ADDING A CATEGORY COLUMN TO GROUP SALARY RANGES 

df_loyalty['Salary Scale'] = pd.cut(df_loyalty['Salary'],bins = 3, labels= ['Low', 'Medium', 'High']) # SETTING CATEGORIES TO GROUP AND COMPARE

In [78]:
# MERGING COLUMNS BASED ON LOYALTY NUMBER, USING A LEFT MERGE IN ORDER TO KEEP ALL THE FLIGHTS, AS THERE ARE MULTIPLE FLIGHTS TO THE SAME PASSENGER.

df = df_flight.merge(df_loyalty, on= 'Loyalty Number', how= 'left')

df.shape # CONFIRMING THE COLUMNS ADDED AND SAME ROW COUNT FROM df_flight

(403760, 26)

In [79]:
# SAVING CLEAN FILES, READY TO BE ANALYZED

df.to_csv("../Files/full_data_clean.csv", index=False)
df_flight.to_csv("../Files/flight_clean.csv", index=False)
df_loyalty.to_csv("../Files/loyalty_clean.csv", index=False)

print('Files are now saved in the folde "'"Files"'", and ready to be analyzed')

Files are now saved in the folde "Files", and ready to be analyzed


In [80]:
# CONFIRMING NULLS ARE AS EXPECTED
df.isnull().sum()

Loyalty Number                      0
Year                                0
Month                               0
Flights Booked                      0
Flights with Companions             0
Total Flights                       0
Distance                            0
Points Accumulated                  0
Points Redeemed                     0
Dollar Cost Points Redeemed         0
Country                             0
Province                            0
City                                0
Postal Code                         0
Gender                              0
Education                           0
Salary                              0
Marital Status                      0
Loyalty Card                        0
CLV                                 0
Enrollment Type                     0
Enrollment Year                     0
Enrollment Month                    0
Cancellation Year              354110
Cancellation Month             354110
Salary Scale                        0
dtype: int64